In [49]:
from scapy.all import *
from scapy.utils import RawPcapReader
from sklearn.model_selection import train_test_split
from tqdm import tqdm


import numpy as np
import pandas as pd
import tensorflow as tf
import keras
import random, os, sys
import glob as glob

In [ ]:
folder = "/Users/michaelwrana/Documents/Courses/Asokan_858/Project/data_large/"

x = []
y = []
for filename in tqdm(glob.iglob(folder + '**', recursive=True)):    
    if os.path.isfile(filename):
        
        # figure out what the label should be based on directory location
        # e.g. data/file1/website1 means file 1 downloaded from website 1
        file_path = filename.split('/')
        file_id = int(file_path[-2][-1])
        
        # load packets from file
        packets = rdpcap(filename)
        
        # configure sequences
        seq_per_file = 10
        seq_len = 25
        packet_sequences = [0] * seq_per_file
        
        # select sequences from packets randomly
        for i in range(seq_per_file):
            start = random.randint(0,len(packets)-seq_len)
            packet_sequence = [raw(packets[j]) for j in range(start, start + seq_len)]
            packet_sequences[i] = [item for sub_list in packet_sequence for item in sub_list]
        
        # assign class labels (based on website ID or file ID)
        file_y = [file_id-1] * seq_per_file
        # file_y = [site_id] * seq_per_file
        
        # append to final dataset
        x = x + packet_sequences
        y = y + file_y
        
        
        
# pad sequences and convert to numpy array
x = tf.keras.preprocessing.sequence.pad_sequences(x)
x = np.asarray(x)
y = np.asarray(y)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.10, random_state=42)

print(len(x_train))
print(len(x_test))

5003it [01:15, 66.24it/s]


45000
5000


In [ ]:
class0_count = 0
class1_count = 0
for item in y_train:
    if item == 0:
        class0_count += 1
    elif item == 1:
        class1_count += 1

print(class0_count)
print(class1_count)

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(128),
    tf.keras.layers.Dense(64),
    tf.keras.layers.Dense(1)
])

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy','AUC','Precision','Recall'])

In [ ]:
model.fit(x_train, y_train, epochs=6)

In [ ]:
results = model.evaluate(x_test, y_test)